In [17]:
import os
import bz2

ROOT = os.path.join(os.path.dirname(os.path.dirname(os.path.abspath(""))), "machine-learning")
DATASET_P = os.path.join(ROOT, "datasets\\spam_classifier")
RAW_DATASET_P = os.path.join(DATASET_P, "raw")
EXTRACTED_DATASET_P = os.path.join(DATASET_P, "extracted")

def extract_dataset_from_bzf(source: str, destination: str):
    if os.listdir(destination) == []:
        print("Dataset has not been extracted")
        print("Extracting...")
        for f in os.listdir(source):
            decompressor = bz2.BZ2Decompressor()
            source_path = os.path.join(source, f)
            extraction_path = os.path.join(destination, f + '.decompressed')
            with open(extraction_path, 'wb') as bzfw, open(source_path, 'rb') as bzfr:
                for data in iter(lambda: bzfr.read(100*1024), b''):
                    bzfw.write(decompressor.decompress(data))
    else:
        print("Data already extracted")

extract_dataset_from_bzf(RAW_DATASET_P, EXTRACTED_DATASET_P)


Dataset has not been extracted
Extracting...
